In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install python-docx
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.6/239.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.4/221.4 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


### Complex2

Goal is to making a meeting summary structured by breakdowns. Each section is a breakdown topic, and the sub-bullet points add more detail about what the issue was, context about it, how it was resolved, and potentially the steps moving forward.

In [ ]:
def get_labeled_meetings(file_path):
  with open(file_path, 'r') as txt_file:
    labeled_meeting_conversations = txt_file.read()
  return labeled_meeting_conversations

In [ ]:
transcript_str = get_labeled_meetings('/content/drive/MyDrive/Coterm/CS224V/complex1_convos_coded_9.10.txt')
print(transcript_str)

NA (NA, NA, NA, 0): [silence, view L0’s lock screen, then his emails]
A1 (Assertive, Describe, Process, 0): guys, I can see we're still waiting. We’re still waiting for a couple of people
A1 (Commissive, Explain, Process, 0): so I just get a cup of coffee real quick, be back
L0 (Assertive, NA, Process, 0): sure
A1 (Commissive, Describe, Process, 0): in two minutes.
NA (NA, NA, NA, 0): [silence] [eventually L0 stops sharing his screen]
A1 (Directive, Negotiate, Process, 0): [sighs or lets out air] ok. So L0 uh there’s one thing I think we should go over in this meeting.
A1 (Assertive, Describe, Process, 0): So the Software A to CMMS integration, all the file, all the stuff that takes time basically has been approved and hopefully it will be done this week.
L0 (Assertive, NA, NA, 0): Oh
A1 (Commissive, Negotiate, Process, 0): So I will, I will start testing on the development system next week
A1 (Assertive, Predict, Process, 0): and then, we should be able to get it onto the production s

In [ ]:
with open('/content/drive/MyDrive/Coterm/CS224V/breakdown_convos_9.10.txt', 'r') as file:
  breakdown_convos_paragraph = file.read()

print(breakdown_convos_paragraph)

A1, L0: and then, we should be able to get it onto the production system, I think, within um one or two months. But the open question is, we need to define some meters somewhere so. kind of need to agree on where we're putting that. Is this the approach temperature? Is it the dp? is it, is it the Software C freezer monitoring, where do we want to start? Or do we just want to start with one thing to check it out? yeah um so this this Software A CMMS integration, we want to use the CMMS to generate the work order right? yeah. So… So. yeah go ahead
A0, A1: How about the installation and who is going to look at the data, interpret the data for us Well interpret the data that's machine learning. That’s why- where it's different than Company R right?wrong and kind of warn you  Company R they one is- even though it's wireless you still have to manually look at the data. Here they will, machine learning will put- it will see when something's  Yeah well without without looking at it does it com

In [ ]:
import os
from openai import OpenAI
from google.colab import userdata

def complex2_summarize(breakdown_convos, labeled_transcript):

  system_message = "You are a meeting summarizer. \
Each line in the labeled meeting conversations is structured as follows:\n\
[Speaker Name] ([Speech Act Classification], [Deepand Classification], [POP Classification], [breakdown exists?]): [speaker utterance]\n\
Speech Acts could be Directive, Commissive, Expressive, Declaration, or Assertive. Definitions:\n\
Directive: Speaker tries to get someone to do something.\n\
Commissive: When someone commits to something.\n\
Expressive: Speaker expresses psychological state about specific affairs.\n\
Declaration: Speaker declares something.\n\
Assertive: Speaker utters soemething to describe an existing state of affairs.\n\n\
Deepand could be Describe, Explain, Evaluate, Predict, Formulate Alternative, Negotiate, or Decide. Definitions:\n\
Describe: Speaker relays what is explicit in a model, documentation, or other source of data or information.\n\
Explain: Speaker provides further insight or clarity into a matter or a position.\n\
Evaluate: Speaker assesses extent to which an option meets defined or inferred requirements, assesses options relative to one another, ranks alternatives.\n\
Predict: Speaker calculates or estimates a parameter value.\n\
Formulate Alternative: Speaker presents an alternative, talks out a possible option modifying the problem space by adding or combining alternate solutions, criteria, or requirements.\n\
Negotiate: Speaker sorts out who will do what or who is responsible for what; focuses on the actor(s) who will do something or who will be responsible for something; figuring out the logistics of someone or a group doing something.\n\
Decide: Speaker makes and/or confirms a choice, chooses an alternative or a solution.\n\n\
POP could be Organization, Process, or Product. Definitions:\n\
Organization: Speaker is talking about an organization.\n\
Process: Speaker is talking about a process.\n\
Product: speaker is talking about a product.\n\n\
Breakdown exists will be 1 when a breakdown exists in this utterance or around it and will be 0 otherwise.\n\n\
Read the transcript and look at speaker turns that voiced a breakdown. \
For each turn with a breakdown, draw context from surrounding utterances and knowledge from the rest of the transcript. \
Think about what the team gained from voicing the breakdown and discussing it. In the instance of misunderstandings, \
explain what understanding was gained about a topic, for example what data is in Software A, or how Software A works. \
Only include an action item sub-bullet point if one was found because of a breakdown or because it was discussed in close proximity to a breakdown.\
Each section should be a breakdown topic and the bullet points should be a detailed discussion about the breakdown in the way explained above. \
Breakdown topics should only be about one discussion. Don't group discussion together unless they are very closely related. \
It is helpful to include speaker names, numbers, statistics, products, and/or entities that were mentioned. \
Provide a structured and detailed discussion based on these instructions. Provided are also conversations that are likely to have \
breakdowns, use them to think about the high-level breakdowns that occurred.\n\
An example bullet point with a sub-bullet point could be:\n\
- A0 reviews his estimates for the cooling tower fan run hours and L0 highlights that he assumes \
1000 hours per year which implies only 3 hours run time per day. This seems low to L0.\n\
    - A0 shows L0 how to bring up the data in Software A.\n\
    - Action item: A0 will schedule a meeting with L0 to continue working on this."

  # input breakdown conversations and labeled transcript into prompt
  prompt = "Breakdown conversations:\n" + breakdown_convos + '\n\nTranscript:\n' + labeled_transcript

  org = userdata.get('org')
  sk_key = userdata.get('sk')
  client = OpenAI(
      organization=org,
      api_key=sk_key
  )

  response = client.chat.completions.create(
      model="gpt-4-1106-preview",
      messages=[
          {"role": "system", "content": system_message},
          {"role": "user", "content": prompt},
      ]
  )

  # Extract the assistant's reply from the response
  assistant_reply = response.choices[0].message.content

  # Print or use the assistant's reply
  print(assistant_reply)
  return assistant_reply

In [ ]:
from docx import Document

# transcript_str = get_labeled_meetings('/content/drive/MyDrive/Coterm/CS224V/complex1_convos_coded_9.10.txt')

# with open('/content/drive/MyDrive/Coterm/CS224V/breakdown_convos_9.10.txt', 'r') as file:
#     breakdown_convos_paragraph = file.read()

summary = complex2_summarize(breakdown_convos_paragraph, transcript_str)
summary = summary.replace('*', '').replace('### ', '').replace('#', '')

doc = Document()
doc.add_paragraph(summary)
doc.save('/content/drive/MyDrive/Coterm/CS224V/final_demo_complex2_9.10_meeting_summary.docx')

### Breakdown: Integration Metrics and Parameters for Software A CMMS

- A1 notes the need to define metrics for integration and poses alternatives such as approach temperature, differential pressure (dp), or Software C freezer monitoring.
  - L0 specifies they want to use CMMS to generate work orders, indicating a key part of the process they need to understand and decide on.
  - A1 outlines that Software A will supply data to CMMS, which can be set to generate work orders or service requests, prompting further discussion on the desired outcome and pointing out the need for a policy decision.

### Breakdown: Machine Learning Data Interpretation and Company Z’s Capabilities

- A1 and A0 discuss the advantages and uncertainties surrounding the use of machine learning to interpret data from the sensors as opposed to Company R's system which requires manual data analysis.
  - L0 introduces the concern about whether Company Z's system comes with actionable recommendations when anomalies ar